# prior belief, measurement update, posterior belief

Assuming that the robot has a map of the world and knows what a door looks like.
- How the robot changes its belief when it senses
- Here is how we model this change of belief https://youtu.be/n1EacrqyCs8

In [28]:
# The prior belief could be modelled with a UNIFORM probability
# distribution over the grid cells
n = 5
p = [1./n for i in range(n)]

In [54]:
# Measurement update
# https://youtu.be/UFcTLCttNRI
# https://youtu.be/SW_wvez0izo

p = [1./n for i in range(n)]

pHit = 0.6
pMiss = 0.2
world = ['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']

# sensor value Z
Z = 'red'
def sense(p, world, Z):
    q = []
    for i, v in enumerate(p):
        hit = (Z == world[i])
        multiplier = hit * pHit + (1-hit) * pMiss
        q.append(v * multiplier)
    q = [v/sum(q) for v in q]
    return q

for z in measurements:
    print([float(f"{x:.4f}") for x in p], f"sum(p) = {sum(p):.2f}" )
    p = sense(p, world, z)
    print([float(f"{x:.4f}") for x in p], f"sum(p) = {sum(p):.2f}" )
    print("".join(['--' for x in range(30)]))
 





[0.2, 0.2, 0.2, 0.2, 0.2] sum(p) = 1.00
[0.1111, 0.3333, 0.3333, 0.1111, 0.1111] sum(p) = 1.00
------------------------------------------------------------
[0.1111, 0.3333, 0.3333, 0.1111, 0.1111] sum(p) = 1.00
[0.2, 0.2, 0.2, 0.2, 0.2] sum(p) = 1.00
------------------------------------------------------------


In [59]:
# Exact motion 

p = [1/9, 1/3, 1/3, 1/9, 1/9]

def move(p, U):
    q = []
    for i in range(len(p)):
        q.append(p[i-U] % len(p))
    return q

q = move(p, 1)
assert q == [1/9, 1/9, 1/3, 1/3, 1/9]

q = move(p, 2)
assert q == [1/9, 1/9, 1/9, 1/3, 1/3]
    

In [62]:
# Inexact motion
p = [0, 1, 0, 0, 0]
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1


def move(p, U):
    q = []
    for i in range(len(p)):
        iExact = (i-U)%len(p)
        iOverShoot = (i-U-1)%len(p)
        iUnderShoot = (i-U+1)%len(p)
        q.append(p[iExact] * pExact + p[iOverShoot] * pOvershoot + p[iUnderShoot] * pUndershoot)   
    
    return q

q = move(p, 1)
print(q)

q = move(p, 2)
print(q)

q = move(p, 42)
print(q)

[0.0, 0.1, 0.8, 0.1, 0.0]
[0.0, 0.0, 0.1, 0.8, 0.1]
[0.0, 0.0, 0.1, 0.8, 0.1]


# Limit distribution

Assuming the initial prior distribution is p = [1, 0, 0, 0, 0]
- what is the stationary distribution when the robot moves infinitely
- https://youtu.be/kfPWiMsnWFI

In [66]:
p = [0, 1, 0, 0, 0]
n = 1000
for i in range(n):
    p = move(p, 1)

print(p)

[0.20000000000000365, 0.20000000000000373, 0.20000000000000365, 0.2000000000000035, 0.2000000000000035]


In [84]:
import math
n = 5
p = [1./n for i in range(n)]

pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1
pHit = 0.6
pMiss = 0.2
world = ['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'red']
motions = [1, 1]

# sensor value Z
Z = 'red'
def sense(p, world, Z):
    q = []
    for i, v in enumerate(p):
        hit = (Z == world[i])
        multiplier = hit * pHit + (1-hit) * pMiss
        q.append(v * multiplier)
    q = [v/sum(q) for v in q]
    return q

def move(p, U):
    q = []
    for i in range(len(p)):
        iExact = (i-U)%len(p)
        iOverShoot = (i-U-1)%len(p)
        iUnderShoot = (i-U+1)%len(p)
        q.append(p[iExact] * pExact + p[iOverShoot] * pOvershoot + p[iUnderShoot] * pUndershoot)   
    
    return q

def print_probs(s, p):
    s2 = ", ".join([f"{x:.4f}" for x in p])
    print(f"{s:30} {s2}, entropy {entropy(p):.4f}")

def entropy(p):
    v = [-x*math.log(x, 10) for x in p]
    return sum(v)

for i in range(len(measurements)):
    print_probs("Before Sensing:", p)
    p = sense(p, world, measurements[i])

    print_probs("After Sensing, Before moving:", p)
    p = move(p, motions[i])

    print_probs("After moving:", p)
    print("".join(['-' for x in range(70)]))

0.0008/(0.0008+0.0999)

Before Sensing:                0.2000, 0.2000, 0.2000, 0.2000, 0.2000, entropy 0.6990
After Sensing, Before moving:  0.1111, 0.3333, 0.3333, 0.1111, 0.1111, entropy 0.6362
After moving:                  0.1111, 0.1333, 0.3111, 0.3111, 0.1333, entropy 0.6549
----------------------------------------------------------------------
Before Sensing:                0.1111, 0.1333, 0.3111, 0.3111, 0.1333, entropy 0.6549
After Sensing, Before moving:  0.0588, 0.2118, 0.4941, 0.1647, 0.0706, entropy 0.5767
After moving:                  0.0788, 0.0753, 0.2247, 0.4329, 0.1882, entropy 0.6112
----------------------------------------------------------------------


0.007944389275074478

In [85]:
0.25/0.3

0.8333333333333334